# 增强版POI爬虫 v2.0 🚀

## 🆕 最新特性 (2025-07-02)
- 🔥 **统一程序入口** - 所有功能集成到一个程序
- 🔇 **完全静默运行** - 屏蔽所有Chrome警告信息
- 📊 **9个增强字段** - 评论数量、电话、网站、营业时间、价格等级
- ⚡ **4倍并发优化** - 智能WebDriver池，高效处理
- 🛡️ **稳定可靠** - 多策略元素定位，智能重试机制
- 💾 **断点续爬** - 自动检查点，支持大规模数据收集
- 🧪 **测试模式** - 快速验证环境和功能
- 📂 **智能文件选择** - 自动扫描并选择最大CSV文件

## 🔧 推荐使用方式
1. **首次使用**: 运行测试模式验证环境
2. **小规模验证**: 处理部分地址确认效果
3. **生产运行**: 开启全量爬取获取完整数据

In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
import time
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 导入增强版爬虫模块
from final_crawler import FinalPOICrawler
from simple_file_selector import SimpleFileSelector, get_simple_file_config

print("📦 所有模块导入成功！")
print(f"📅 当前时间: {pd.Timestamp.now()}")

## 📋 1. 配置设置

In [ ]:
# 🔥 新功能：智能文件选择器
ENABLE_FILE_SELECTION = True  # 设置为True启用文件选择，False使用默认文件

if ENABLE_FILE_SELECTION:
    print("📂 启动智能文件选择器...")
    print("=" * 50)
    
    # 使用简化的文件选择器
    file_config = get_simple_file_config(suffix="poi_notebook")
    
    if file_config['has_input']:
        INPUT_FILE = file_config['input_file']
        OUTPUT_FILE = file_config['output_file']
        
        print(f"✅ 文件选择完成:")
        print(f"📥 输入文件: {INPUT_FILE}")
        print(f"📤 输出文件: {OUTPUT_FILE}")
        
        # 显示文件信息
        try:
            df_check = pd.read_csv(INPUT_FILE)
            print(f"📊 文件信息: {len(df_check):,} 行, 列: {list(df_check.columns)}")
        except Exception as e:
            print(f"⚠️ 无法读取文件信息: {e}")
            
    else:
        print("❌ 未找到可用的CSV文件，使用默认配置")
        INPUT_FILE = 'data/input/千代田区_complete_1751433587.csv'
        OUTPUT_FILE = 'data/output/千代田区_poi_notebook.csv'

else:
    # 使用默认文件
    print("📄 使用默认文件配置")
    INPUT_FILE = 'data/input/千代田区_complete_1751433587.csv'
    OUTPUT_FILE = 'data/output/千代田区_poi_notebook.csv'

print(f"\n📋 最终配置:")
print(f"  输入: {INPUT_FILE}")
print(f"  输出: {OUTPUT_FILE}")

## 📂 1. 文件选择

In [3]:
# 🔥 动态配置 - 基于文件选择结果
CONFIG = {
    'max_workers': 4,          # 并发线程数
    'driver_pool_size': 4,     # WebDriver池大小
    'batch_size': 15,          # 批量保存数据量
    'timeout': 12,             # 页面加载超时时间(秒)
    'retry_times': 2,          # 重试次数
    'headless': True,          # 🔥 强制无头模式 - 不显示Chrome窗口
    'checkpoint_interval': 30, # 检查点保存间隔
    'input_file': INPUT_FILE,  # 🔥 使用选择的输入文件
    'output_file': OUTPUT_FILE # 🔥 使用生成的输出文件
}

print("⚙️ 最终运行配置:")
for key, value in CONFIG.items():
    icon = "🔥" if key in ['headless', 'input_file', 'output_file'] else "📝"
    print(f"  {icon} {key}: {value}")

⚙️ 最终运行配置:
  📝 max_workers: 4
  📝 driver_pool_size: 4
  📝 batch_size: 15
  📝 timeout: 12
  📝 retry_times: 2
  🔥 headless: True
  📝 checkpoint_interval: 30
  🔥 input_file: data/input/千代田区_complete_1751433587.csv
  🔥 output_file: data\output\千代田区_1751433587_poi_notebook_20250702_1632.csv


## 📂 2. 数据预览

In [4]:
# 读取并预览输入数据
try:
    df_input = pd.read_csv(CONFIG['input_file'])
    print(f"📄 成功读取输入文件: {CONFIG['input_file']}")
    print(f"📊 数据统计:")
    print(f"  总行数: {len(df_input):,}")
    print(f"  列名: {list(df_input.columns)}")
    
    print(f"\n📋 数据预览:")
    display(df_input.head())
    
    # 准备地址列表
    addresses = df_input['Address'].dropna().tolist()
    print(f"\n📍 有效地址数量: {len(addresses):,}")
    print(f"📍 示例地址:")
    for i, addr in enumerate(addresses[:3], 1):
        print(f"  {i}. {addr}")
    
except Exception as e:
    print(f"❌ 读取输入文件失败: {e}")
    addresses = []

📄 成功读取输入文件: data/input/千代田区_complete_1751433587.csv
📊 数据统计:
  总行数: 8,693
  列名: ['District', 'Latitude', 'Longitude', 'Address']

📋 数据预览:


,District,Latitude,Longitude,Address
0,千代田区,35.690357,139.771265,東京都千代田区鍛冶町1丁目7-1
1,千代田区,35.686220,139.734724,東京都千代田区二番町10-46
2,千代田区,35.695114,139.762307,東京都千代田区神田小川町3丁目6-2
3,千代田区,35.690443,139.773287,東京都千代田区鍛冶町1丁目10-7
4,千代田区,35.703297,139.770201,東京都千代田区外神田6丁目5-9



📍 有效地址数量: 8,693
📍 示例地址:
  1. 東京都千代田区鍛冶町1丁目7-1
  2. 東京都千代田区二番町10-46
  3. 東京都千代田区神田小川町3丁目6-2


## 🧪 3. 小规模测试 (可选)

In [ ]:
# 测试前5个地址
TEST_MODE = True  # 设置为False跳过测试，直接全量运行
TEST_COUNT = 5

if TEST_MODE and addresses:
    print(f"🧪 测试模式: 处理前 {TEST_COUNT} 个地址")
    
    test_config = CONFIG.copy()
    test_config['output_file'] = 'data/output/测试结果_poi.csv'
    test_config['max_workers'] = 2  # 测试时减少并发
    
    test_addresses = addresses[:TEST_COUNT]
    
    print(f"🚀 开始测试爬取...")
    start_time = time.time()
    
    # 清理测试模式的检查点
    checkpoint_file = Path('checkpoint.json')
    if checkpoint_file.exists():
        checkpoint_file.unlink()
        print("🧹 已清理检查点文件")
    
    crawler = FinalPOICrawler(test_config)
    try:
        crawler.process_addresses(test_addresses)
        
        elapsed = time.time() - start_time
        print(f"\n⏱️ 测试完成，耗时: {elapsed:.1f} 秒")
        print(f"📈 平均速度: {elapsed/TEST_COUNT:.1f} 秒/个")
        
        # 查看测试结果
        if Path(test_config['output_file']).exists():
            test_results = pd.read_csv(test_config['output_file'])
            print(f"\n📊 测试结果统计:")
            print(f"  总POI数: {len(test_results)}")
            
            if len(test_results) > 0:
                print(f"\n📋 增强数据字段预览:")
                display(test_results.head())
                
                print(f"\n📈 数据质量统计:")
                print(f"  有评分的POI: {test_results['rating'].notna().sum()}")
                print(f"  有评论数的POI: {test_results['review_count'].notna().sum()}")
                print(f"  有电话的POI: {test_results['phone'].notna().sum()}")
                print(f"  有网站的POI: {test_results['website'].notna().sum()}")
        
    except Exception as e:
        print(f"❌ 测试失败: {e}")
    finally:
        crawler.close()
        
else:
    print("⏭️ 跳过测试模式")

## 🚀 4. 全量生产爬取

In [ ]:
# 确认是否执行全量爬取
RUN_FULL_CRAWL = False  # 设置为True开始全量爬取

if RUN_FULL_CRAWL and addresses:
    print(f"🎯 开始全量POI爬取")
    print(f"📊 预计处理 {len(addresses):,} 个地址")
    print(f"⚡ 性能设置: {CONFIG['max_workers']} 并发线程，无头模式")
    
    estimated_time = len(addresses) * 2.5 / CONFIG['max_workers'] / 60  # 估算时间
    print(f"⏱️ 预计耗时: {estimated_time:.1f} 分钟")
    
    confirmation = input("\n确认开始全量爬取？(输入 'yes' 确认): ")
    
    if confirmation.lower() == 'yes':
        print(f"\n🚀 开始全量爬取...")
        start_time = time.time()
        
        crawler = FinalPOICrawler(CONFIG)
        try:
            crawler.process_addresses(addresses)
            
            elapsed = time.time() - start_time
            print(f"\n🎉 全量爬取完成！")
            print(f"⏱️ 总耗时: {elapsed/60:.1f} 分钟")
            print(f"📈 平均速度: {elapsed/len(addresses):.1f} 秒/个")
            
        except KeyboardInterrupt:
            print("\n⏹️ 用户中断爬取")
        except Exception as e:
            print(f"\n❌ 爬取过程出错: {e}")
        finally:
            crawler.close()
    else:
        print("❌ 已取消全量爬取")
        
else:
    print("⏭️ 跳过全量爬取")
    print("💡 如需运行全量爬取，请将 RUN_FULL_CRAWL 设置为 True")

## 📊 5. 结果分析

In [ ]:
# 分析爬取结果
output_file = CONFIG['output_file']

if Path(output_file).exists():
    results = pd.read_csv(output_file)
    
    print(f"📊 结果统计报告")
    print(f"="*50)
    print(f"📄 输出文件: {output_file}")
    print(f"📈 总POI数量: {len(results):,}")
    print(f"🏢 唯一建筑物: {results['blt_name'].nunique():,}")
    print(f"⭐ 平均评分: {results['rating'].mean():.2f}")
    print(f"💬 平均评论数: {results['review_count'].mean():.0f}")
    
    print(f"\n📋 数据完整性分析:")
    completeness = {
        '名称': results['name'].notna().sum(),
        '评分': results['rating'].notna().sum(), 
        '评论数': results['review_count'].notna().sum(),
        '类别': results['category'].notna().sum(),
        '地址': results['address'].notna().sum(),
        '电话': results['phone'].notna().sum(),
        '网站': results['website'].notna().sum(),
        '营业时间': results['hours'].notna().sum(),
        '价格等级': results['price_level'].notna().sum()
    }
    
    for field, count in completeness.items():
        percentage = count / len(results) * 100
        print(f"  {field}: {count:,} ({percentage:.1f}%)")
    
    print(f"\n🏆 TOP 10 建筑物 (按POI数量):")
    top_buildings = results.groupby('blt_name').size().sort_values(ascending=False).head(10)
    for i, (building, count) in enumerate(top_buildings.items(), 1):
        print(f"  {i:2d}. {building}: {count} 个POI")
    
    print(f"\n⭐ 评分分布:")
    rating_dist = results['rating'].value_counts().sort_index()
    for rating, count in rating_dist.head().items():
        print(f"  {rating}星: {count} 个POI")
    
    print(f"\n📱 联系方式统计:")
    print(f"  有电话号码: {results['phone'].notna().sum()} 个POI")
    print(f"  有官方网站: {results['website'].notna().sum()} 个POI")
    
    print(f"\n💰 价格等级分布:")
    if 'price_level' in results.columns:
        price_dist = results['price_level'].value_counts().sort_index()
        price_labels = {1: '$ (便宜)', 2: '$$ (中等)', 3: '$$$ (偏贵)', 4: '$$$$ (昂贵)'}
        for level, count in price_dist.items():
            label = price_labels.get(level, f'{level}级')
            print(f"  {label}: {count} 个POI")
    
    print(f"\n📋 数据样例:")
    display(results.head())
    
else:
    print(f"❌ 输出文件不存在: {output_file}")

## 📈 6. 数据导出和后处理

In [ ]:
# 数据清理和导出
if Path(output_file).exists():
    results = pd.read_csv(output_file)
    
    print(f"🔧 数据后处理...")
    
    # 数据清理
    cleaned_results = results.copy()
    
    # 移除重复POI
    before_dedup = len(cleaned_results)
    cleaned_results = cleaned_results.drop_duplicates(subset=['name', 'blt_name'], keep='first')
    after_dedup = len(cleaned_results)
    print(f"  🗑️ 移除重复POI: {before_dedup - after_dedup} 个")
    
    # 数据类型优化
    if 'rating' in cleaned_results.columns:
        cleaned_results['rating'] = pd.to_numeric(cleaned_results['rating'], errors='coerce')
    if 'review_count' in cleaned_results.columns:
        cleaned_results['review_count'] = pd.to_numeric(cleaned_results['review_count'], errors='coerce')
    if 'price_level' in cleaned_results.columns:
        cleaned_results['price_level'] = pd.to_numeric(cleaned_results['price_level'], errors='coerce')
    
    # 保存清理后的数据
    clean_output_file = output_file.replace('.csv', '_清理版.csv')
    cleaned_results.to_csv(clean_output_file, index=False, encoding='utf-8')
    print(f"  💾 清理版数据已保存: {clean_output_file}")
    
    # 按建筑物分组导出
    building_summary = cleaned_results.groupby('blt_name').agg({
        'name': 'count',
        'rating': 'mean',
        'review_count': 'sum',
        'category': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'Unknown'
    }).round(2)
    
    building_summary.columns = ['POI数量', '平均评分', '总评论数', '主要类别']
    building_summary = building_summary.sort_values('POI数量', ascending=False)
    
    summary_file = output_file.replace('.csv', '_建筑物汇总.csv')
    building_summary.to_csv(summary_file, encoding='utf-8')
    print(f"  📊 建筑物汇总已保存: {summary_file}")
    
    print(f"\n✅ 数据处理完成！")
    print(f"📁 输出文件:")
    print(f"  - 原始数据: {output_file}")
    print(f"  - 清理版本: {clean_output_file}")
    print(f"  - 建筑汇总: {summary_file}")
    
else:
    print(f"❌ 没有找到结果文件进行后处理")

## 🔍 7. 检查点状态

In [ ]:
# 查看检查点信息
checkpoint_file = 'checkpoint.json'

if Path(checkpoint_file).exists():
    try:
        with open(checkpoint_file, 'r', encoding='utf-8') as f:
            checkpoint = json.load(f)
        
        print(f"📋 检查点状态报告")
        print(f"="*40)
        print(f"⏰ 最后更新: {checkpoint.get('timestamp', 'Unknown')}")
        print(f"📊 已处理地址: {checkpoint.get('processed_count', 0):,}")
        print(f"✅ 成功数量: {checkpoint.get('success_count', 0):,}")
        print(f"❌ 失败数量: {len(checkpoint.get('failed_addresses', [])):,}")
        
        if checkpoint.get('processed_count', 0) > 0:
            success_rate = checkpoint.get('success_count', 0) / checkpoint.get('processed_count', 1) * 100
            print(f"📈 成功率: {success_rate:.1f}%")
        
        failed_addresses = checkpoint.get('failed_addresses', [])
        if failed_addresses:
            print(f"\n❌ 失败地址样例 (前5个):")
            for i, addr in enumerate(failed_addresses[:5], 1):
                print(f"  {i}. {addr}")
                
    except Exception as e:
        print(f"❌ 读取检查点文件失败: {e}")
        
else:
    print(f"ℹ️ 没有检查点文件")

## 🎯 v2.0 更新总结

### 🚀 性能提升
- **静默运行**: 完全屏蔽Chrome启动警告，纯净输出体验
- **4倍并发**: 智能WebDriver池，同时处理4个地址
- **内存优化**: 禁用图片、插件等，专注数据提取
- **智能等待**: 替代固定延时，提高爬取效率

### 📊 数据增强  
- **9个字段**: name, rating, review_count, category, address, phone, website, hours, price_level
- **元数据完整**: 建筑物名称、坐标、爬取时间等
- **数据质量**: 支持多种POI类型，智能去重

### 🛡️ 稳定可靠
- **多策略定位**: 页面结构变化也能正常工作
- **智能重试**: 自动处理临时网络错误
- **断点续爬**: 支持大规模数据收集，意外中断可恢复
- **错误容错**: 单个地址失败不影响整体进度

### 🔧 易用性改进
- **统一入口**: 一个程序包含所有功能
- **智能文件选择**: 自动扫描并选择最大CSV文件
- **测试模式**: 快速验证环境和配置
- **详细进度**: 实时显示爬取进度和统计信息

---
**💡 提示**: 
- 首次使用建议先运行测试模式验证环境
- 大规模爬取前建议小批量测试确认效果
- 生成的CSV文件包含完整的9个数据字段
- 程序支持中断后断点续爬，适合长时间运行